In [2]:
import logging
import sys
import os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [3]:
import os
os.environ['NUMEXPR_MAX_THREADS'] = '4'
os.environ['NUMEXPR_NUM_THREADS'] = '2'
import numexpr as ne

In [4]:
import os
os.environ["OPENAI_API_KEY"] = "sk-xxx"
import openai
openai.api_key = "sk-xxx"

In [7]:
from llama_index import SimpleDirectoryReader
documents = SimpleDirectoryReader('assets/AndrewHuberman/sleep', filename_as_id=True).load_data()

In [ ]:
print(documents)

In [ ]:
import chromadb
chroma_client = chromadb.EphemeralClient()
chroma_collection = chroma_client.create_collection("andrew_sleep_db")

In [8]:
from llama_index.vector_stores import ChromaVectorStore
from llama_index import VectorStoreIndex
from llama_index.storage.storage_context import StorageContext

In [11]:
db = chromadb.PersistentClient(path="./storage/chroma")
chroma_collection = db.get_or_create_collection("andrew_sleep_db")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_vector_store(vector_store=vector_store, storage_context=storage_context)

INFO:chromadb.telemetry.posthog:Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.
Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.


In [19]:
from llama_index.embeddings import OpenAIEmbedding
embed_model = OpenAIEmbedding()
search_text = "something"
embedding = embed_model.get_text_embedding(search_text)

results = chroma_collection.query(
    query_embeddings=[embedding],
    n_results=2
)
print(results)

{'ids': [['65df8d15-eb4a-4f82-bc7f-1cb2996bd798', '333f7c97-50e4-4991-a48e-f0342a661b8e']], 'distances': [[0.3660924434661865, 0.36859169602394104]], 'metadatas': [[{'_node_content': '{"id_": "65df8d15-eb4a-4f82-bc7f-1cb2996bd798", "embedding": null, "metadata": {}, "excluded_embed_metadata_keys": [], "excluded_llm_metadata_keys": [], "relationships": {"1": {"node_id": "assets/AndrewHuberman/sleep/115_Dr_Gina_Poe_Use_Sleep_to_Enhance_Learning_Memory_&_Emotional_State_Huberman_Lab_Podcast.txt", "node_type": null, "metadata": {}, "hash": "fbcacda02d943cacab9cdd5d314b1865ca59a36997b288e52d3c59fd535a314b"}, "2": {"node_id": "fddaa338-a559-456a-854b-df1513c36dde", "node_type": null, "metadata": {}, "hash": "76e97c80f2973fe2beacba3e84b1bfc1c9fe6b1b1df035303bdfee8010e39604"}, "3": {"node_id": "dc4177bc-aba5-4d0d-bc52-62bea388e2fd", "node_type": null, "metadata": {}, "hash": "74bee07104321e80ae759823e569544910bfb938408b9c28f46ea1d72efba2fe"}}, "hash": "9e2b0e1383a0735117a5b6dd709d8026a0b568c8f

In [16]:
query_engine = index.as_query_engine()

In [18]:
response = query_engine.query("How does temperature affect sleep?")
print(response)

Temperature can have a significant impact on sleep. Lowering the temperature in the sleeping environment, either by adjusting the room temperature or using a controllable mattress cover, can naturally make the sleep environment cooler. This can help in falling asleep and staying asleep. If a person feels too warm under the blankets, they can extend a hand or foot out from under the blankets to cool off, as the palms of the hands, the upper half of the face, and the bottoms of the feet have special portals for passing heat. On the other hand, if the room is too warm, it can be difficult to cool off, and pushing off the blankets may not be sufficient. It is generally recommended to sleep in a relatively cool or cold environment and layer on blankets as needed. Additionally, lowering core body temperature by taking a hot shower or bath and then cooling off can help in falling asleep.
